In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed
import pandas as pd
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch,Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import proj3d

plt.rcParams["figure.figsize"] = [10, 10]

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt

from scipy.integrate import quad,dblquad

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


## Announcements

  - Quiz 7 this week. 
    - (Rectangular) triple integrals
    - Center of mass/moments of inertia.
  - Homework 10 due Tuesday.
  - Exam 2 04/09
    - Through HW10
    - Practice materials in the "review_materials" folder on Canvas.
    - [Screencast with old exam here](https://youtu.be/d2n3ytcJ5j4).
    
    

# One-minute Review

Fubini's Theorem applies directly to triple integrals.

$$\iiint_E f(x,y,z)\,dV = \int_a^b \int_{g(x)}^{h(x)} \int_{j(x,y)}^{k(x,y)} f(x,y,z)\,dz\,dy\,dx $$

In setting up such an integral, look for one coordinate whose upper- and lower-bounds are easily expressed as functions of the other two variables. Make this the "inside" variable. This reduces the problem to a 2-D case (easier said than done).
  

# Lecture 17

  - Objectives
    - Triple Integration
      - Integrals in Spherical and Cylindrical Coords
      - Integration Review
    - Midterm 2 review.
  - Resources
    - Content
      - Stewart: §15.5–7
      - New Strang:  
        - [§5.4](https://openstax.org/books/calculus-volume-3/pages/5-4-triple-integrals) Some good practice on changing order of integration here, Particularly, nos. 203–206, 211, 212, 215–220, 235, 240, though some answers may have errors.
        - [§5.5](https://openstax.org/books/calculus-volume-3/pages/5-5-triple-integrals-in-cylindrical-and-spherical-coordinates)
      - [Slides](https://hub.columbiajupyter2.org/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdrewyoungren%2Fmvc-f19&urlpath=tree%2Fmvc-f19%2Fslides%2Fmvc-L17.ipynb) via JupyterHub
    - Practice
      - Mooculus: [Cylindrical Coordinates](https://ximera.osu.edu/mooculus/calculus3/commonCoordinates/digInCylindricalCoordinates) [Spherical](https://ximera.osu.edu/mooculus/calculus3/computationsAndInterpretations/digInMassMomentsAndCenterOfMass)
    - Extras
      - CalcBLUE: [Cylindrical Coordinates](https://www.youtube.com/watch?v=YIi7muLgI9U&list=PL8erL0pXF3JaJdUcmc_PeGV-vG5z87BkD&index=90) [Spherical Coordinates](https://www.youtube.com/watch?v=Zc8uCT-e5KI&list=PL8erL0pXF3JaJdUcmc_PeGV-vG5z87BkD&index=98) 

# Volumes

When integrating, we divide space up by taking small intervals in each coordinate. Let's look at the volumes $\Delta V$ of such subdivisions.

In [1]:
@interact(angle=(-108,66,6),mov=(.1,1,.05))
def _(angle=-24,vangle=(0,90,6),coords=['rectangular','cylindrical','spherical'],mov=1):
#     a,b,c = (.5,.6,.8)
#     r = sqrt(a**2 + b**2)
#     th = np.arctan(b/a)
#     rho = sqrt(a**2 + b**2 + c**2)
#     phi = np.arctan(r/c)
    fig = plt.figure(figsize=(10,10))
    ax= fig.add_subplot(111,projection='3d')
    u = np.linspace(0,1,81)
    v = np.linspace(0,1,81)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V
    Z = np.ones_like(U)
    if coords=='rectangular':
        ax.plot_wireframe(U,V,0*Z+mov,rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe(U,V,0*Z+mov-.1,rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([i,i],[j,j],[mov-.1,mov],color='gray',alpha=.7)
    elif coords=='cylindrical':
        ax.plot_wireframe(mov*cos(pi/2*V),mov*sin(pi/2*V),U,rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe((mov-.1)*cos(pi/2*V),(mov-.1)*sin(pi/2*V),U,rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([(mov-.1)*cos(pi/2*i),(mov)*cos(pi/2*i)],[(mov-.1)*sin(pi/2*i),mov*sin(pi/2*i)],[j,j],color='gray',alpha=.7)

    elif coords=='spherical':
        ax.plot_wireframe(mov*sin(pi/2*U)*cos(pi/2*V),mov*sin(pi/2*U)*sin(pi/2*V),mov*cos(pi/2*U),rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe((mov-.1)*sin(pi/2*U)*cos(pi/2*V),(mov-.1)*sin(pi/2*U)*sin(pi/2*V),(mov-.1)*cos(pi/2*U),rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([(mov-.1)*sin(pi/2*j)*cos(pi/2*i),(mov)*sin(pi/2*j)*cos(pi/2*i)],[(mov-.1)*sin(pi/2*j)*sin(pi/2*i),(mov)*sin(pi/2*j)*sin(pi/2*i)],[(mov-.1)*cos(pi/2*j),(mov)*cos(pi/2*j)],color='gray',alpha=.7)


    ax.view_init(vangle,angle)
    
    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([0,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

NameError: name 'interact' is not defined

$$dV=dx\,dy\,dz = r\,dr\,d\theta\,dz = \rho^2 \sin \phi \,d\rho\,d\theta\,d\phi$$

# Exam 2 Review

  - Administered via Gradescope.
  - Not cumulative in a direct sense (cumulative as necessary).
  - Topics
    - Chain Rule
    - Gradients
      - Definition
      - Properties
      - Directional Derivative
    - Optimization
      - Critical Points
      - Classifying critical points
      - Unconstrained optimization
      - Lagrange Multipliers
    - Integration
      - Riemann sums
      - Double integrals
      - Fubini's Theorem
      - Polar Coordinates
      - Applications
        - Mass/density
        - Center of mass/centroids
        - Moments of inertia
        - Expected value/probability
      - Triple integrals
      - Cylindrical coordinates
      - Spherical Coordinates

### Exercise

Let $E$ be the region in $\RR^3$ defined by
\begin{align*}
& 0\leq x^2+y^2+z^2\leq 1,\\
& x\geq0.
\end{align*}

Which of the following integrals must be $0$? (Do **not** compute them. Use other means.)

  1. $\displaystyle\iiint_E \sin(x) dV$.
  1.  $\displaystyle\iiint_E xy^2z^2 dV$. 
  1.  $\displaystyle\iiint_E x^2yz dV$.
  1.  $\displaystyle\iiint_E \cos(y) dV$. 
  1.  None of the above.

Which region in $\RR^3$ described below has the smallest volume?


  1. All cylindrical coordinates $r,\theta,z$ are between 0 and 1. 
  1. All rectangular coordinates $x,y,z$ are between 0 and 1. 
  1. All spherical coordinates $\rho,\theta,\phi$ are between 0 and 1.
  1. All of the above have equal volume. 
  1. Don't choose this.
